In [16]:
import requests
import pandas as pd

# API_KEY = "YOUR_API_KEY"
device_id = "72537f19-35c1-4ad2-a209-efeb67f400fc"
API_KEY = "fyRy8244VHbZ4McpJ4KO3hRx4n4JYOmOTQuVVzdBsWYzqVS7"
BASE_URL = "https://api.kaiterra.com/v1"

In [24]:
import requests
import pandas as pd

device_id = "72537f19-35c1-4ad2-a209-efeb67f400fc"
BASE_URL = "https://api.kaiterra.com/v1"

# Call the /top endpoint
url = f"{BASE_URL}/devices/{device_id}/top?key={API_KEY}"
resp = requests.get(url)

print("Status:", resp.status_code)

data = resp.json()

# Flatten into DataFrame
rows = []
for p in data.get("data", []):
    rows.append({
        "parameter": p.get("param"),
        "value": p["points"][0]["value"] if p.get("points") else None,
        "timestamp": p["points"][0]["ts"] if p.get("points") else None,
        "units": p.get("units")
    })

df_live = pd.DataFrame(rows)

# Convert timestamp to datetime
df_live["timestamp"] = pd.to_datetime(df_live["timestamp"])

print(df_live)

Status: 200
  parameter   value                 timestamp  units
0      rco2  503.20 2025-11-24 07:41:00+00:00    ppm
1    rhumid   67.03 2025-11-24 07:41:00+00:00      %
2    rpm10c    0.95 2025-11-24 07:41:00+00:00  µg/m³
3    rpm25c    0.95 2025-11-24 07:41:00+00:00  µg/m³
4     rtemp   22.74 2025-11-24 07:41:00+00:00      C
5     rtvoc  165.50 2025-11-24 07:41:00+00:00    ppb
6      tvoc  165.50 2025-11-24 07:41:00+00:00    ppb


In [25]:
df_live.to_csv("realtime.csv")

In [20]:
import requests
import datetime
import pandas as pd

# API_KEY = "YOUR_API_KEY"   # <-- put your real API key here
device_id = "72537f19-35c1-4ad2-a209-efeb67f400fc"

BASE_URL = "https://api.kaiterra.com/v1"

# Time range: last 24 hours
end = datetime.datetime.utcnow()
begin = end - datetime.timedelta(hours=24)

params = {
    "key": API_KEY,
    "begin": begin.isoformat() + "Z",
    "end": end.isoformat() + "Z"
}

# Request historical data
url = f"{BASE_URL}/devices/{device_id}/history"
resp = requests.get(url, params=params)

print("Status:", resp.status_code)
print("Raw Response:", resp.text)   # helpful for debugging

# Convert to DataFrame only if JSON returned
if resp.status_code == 200:
    history = resp.json()

    rows = []
    for p in history.get("data", []):
        param = p["param"]
        units = p.get("units", None)

        for point in p["points"]:
            rows.append({
                "timestamp": point["ts"],
                "parameter": param,
                "value": point["value"],
                "units": units
            })

    df = pd.DataFrame(rows)

    # Convert timestamp to datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    print("\nPreview of historical data:")
    print(df.head())

else:
    print("\n⚠️ History request failed. This usually means your API key does NOT have historical data permissions.")

/tmp/ipython-input-1633317452.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.datetime.utcnow()


Status: 403
Raw Response: {"errors":[{"status":"403","title":"error","details":"Your client is unauthorized to call this API"}]}

⚠️ History request failed. This usually means your API key does NOT have historical data permissions.
